In [43]:
import pandas as pd
# from yahoo_fin import stock_info as si
import yfinance as yf
# import streamlit as st
import numpy as np
from matplotlib import pyplot as plt

def comma_format(number):
    if not pd.isna(number) and number != 0:
        return '{:,.0f}'.format(number)

def percentage_format(number):
    if not pd.isna(number) and number != 0:
        return '{:.1%}'.format(number) 

def calculate_value_distribution(parameter_dict_1, parameter_dict_2, parameter_dict_distribution):
    parameter_list = []
    parameter_list.append(parameter_dict_1['latest revenue'])
    for i in parameter_dict_2:
        if parameter_dict_distribution[i] == 'normal':
            parameter_list.append((np.random.normal(parameter_dict_1[i], parameter_dict_2[i]))/100)
        if parameter_dict_distribution[i] == 'triangular':
            lower_bound = parameter_dict_1[i]
            mode = parameter_dict_2[i]
            parameter_list.append((np.random.triangular(lower_bound, mode, 2*mode-lower_bound))/100)
        if parameter_dict_distribution[i] == 'uniform':
            parameter_list.append((np.random.uniform(parameter_dict_1[i], parameter_dict_2[i]))/100)
    parameter_list.append(parameter_dict_1['net debt'])
    return parameter_list

class Company:

    def __init__(self, ticker):
        self.company = yf.Ticker(ticker)
        self.income_statement = self.company.income_stmt
        self.balance_sheet = self.company.balance_sheet
        self.cash_flow_statement = self.company.cashflow
        self.inputs = self.get_inputs_df()

    def get_inputs_df(self):
        income_statement_list = ['Total Revenue', 'EBIT', 
        'Pretax Income', 'Tax Provision'
        ]
        balance_sheet_list = ['Current Assets', 'Cash And Cash Equivalents',
        'Current Liabilities', 'Current Debt',
        'Long Term Debt'
        ]
        balance_sheet_list_truncated = ['Current Assets', 'Cash And Cash Equivalents',
        'Current Liabilities', 'Long Term Debt'
        ]
        balance_sheet_list_no_debt = ['Current Assets', 'Cash And Cash Equivalents',
        'Current Liabilities'
        ]

        cash_flow_statement_list = ['Depreciation And Amortization', 
        'Capital Expenditure'
        ]
        
        income_statement_df = self.income_statement[self.income_statement.index.isin(income_statement_list)]
        try:
            balance_sheet_df = self.balance_sheet[self.balance_sheet.index.isin(balance_sheet_list)]
        except KeyError:
            try:
                balance_sheet_df = self.balance_sheet[self.balance_sheet.index.isin(balance_sheet_list_truncated)]
            except KeyError:
                balance_sheet_df = self.balance_sheet[self.balance_sheet.index.isin(balance_sheet_list_no_debt)]
        cash_flow_statement_df = self.cash_flow_statement[self.cash_flow_statement.index.isin(cash_flow_statement_list)]

        df = pd.concat([income_statement_df, balance_sheet_df])
        df = pd.concat([df, cash_flow_statement_df])
    
        columns_ts = df.columns
        columns_str = [str(i)[:10] for i in columns_ts]
        columns_dict = {}
        for i,f in zip(columns_ts, columns_str):
            columns_dict[i] = f
        df.rename(columns_dict, axis = 'columns', inplace = True)

        columns_str.reverse()
        df = df[columns_str]
        
        prior_revenue_list = [None]
        for i in range(len(df.loc['Total Revenue'])):
            if i != 0 and i != len(df.loc['Total Revenue']):
                prior_revenue_list.append(df.loc['Total Revenue'][i-1])

        df.loc['priorRevenue'] = prior_revenue_list
        df.loc['revenueGrowth'] = (df.loc['Total Revenue'] - df.loc['priorRevenue']) / df.loc['priorRevenue']
        df.loc['ebitMargin'] = df.loc['EBIT']/df.loc['Total Revenue'] 
        df.loc['taxRate'] = df.loc['Tax Provision']/df.loc['Pretax Income'] 
        df.loc['netCapexOverSales'] = (- df.loc['Capital Expenditure'] - df.loc['Depreciation And Amortization']) / df.loc['Total Revenue']
        try:
            df.loc['nwc'] = (df.loc['Current Assets'] - df.loc['Cash And Cash Equivalents']) - (df.loc['Current Liabilities'] - df.loc['Current Debt'])
        except KeyError:
            df.loc['nwc'] = (df.loc['Current Assets'] - df.loc['Cash And Cash Equivalents']) - (df.loc['Current Liabilities'])
        df.loc['nwcOverSales'] = df.loc['nwc']/df.loc['Total Revenue']
        try:
            df.loc['netDebt'] = df.loc['Current Debt'] + df.loc['Long Term Debt'] - df.loc['Cash And Cash Equivalents']
        except KeyError:
            try:
                df.loc['netDebt'] = df.loc['Long Term Debt'] - df.loc['Cash And Cash Equivalents']
            except KeyError:
                df.loc['netDebt'] = - df.loc['Cash And Cash Equivalents']
        df = df[12:len(df)].drop('nwc')
        df['Historical average'] = [df.iloc[i].mean() for i in range(len(df))]
        return df
        # return self.income_statement, self.balance_sheet, self.cash_flow_statement

    def get_free_cash_flow_forecast(self, parameter_list):
        df = pd.DataFrame(columns = [1, 2, 3, 4, 5])
        revenue_list = []
        for i in range(5):
            revenue_list.append(parameter_list[0] * (1 + parameter_list[1]) ** (i+1))
        df.loc['Revenues'] = revenue_list
        ebit_list = [i * parameter_list[2] for i in df.loc['Revenues']]
        df.loc['EBIT'] = ebit_list
        tax_list = [i * parameter_list[3] for i in df.loc['EBIT']]
        df.loc['Taxes'] = tax_list
        nopat_list = df.loc['EBIT'] - df.loc['Taxes']
        df.loc['NOPAT'] = nopat_list
        net_capex_list = [i * parameter_list[4] for i in df.loc['Revenues']]
        df.loc['Net capital expenditures'] = net_capex_list
        nwc_list = [i * parameter_list[5] for i in df.loc['Revenues']]
        df.loc['Changes in NWC'] = nwc_list
        free_cash_flow_list = df.loc['NOPAT'] - df.loc['Net capital expenditures'] - df.loc['Changes in NWC']
        df.loc['Free cash flow'] = free_cash_flow_list
        return df

    def discount_free_cash_flows(self, parameter_list, discount_rate, terminal_growth):
        free_cash_flow_df = self.get_free_cash_flow_forecast(parameter_list)
        df = free_cash_flow_df
        discount_factor_list = [(1 + discount_rate) ** i for i in free_cash_flow_df.columns]
        df.loc['Discount factor'] = discount_factor_list
        present_value_list = df.loc['Free cash flow'] / df.loc['Discount factor']
        df.loc['PV free cash flow'] = present_value_list
        df[0] = [0 for i in range(len(df))]
        df.loc['Sum PVs', 0] = df.loc['PV free cash flow', 1:5].sum()
        df.loc['Terminal value', 5] = df.loc['Free cash flow', 5] * (1 + terminal_growth) / (discount_rate - terminal_growth)
        df.loc['PV terminal value', 0] = df.loc['Terminal value', 5] / df.loc['Discount factor', 5]
        df.loc['Company value (enterprise value)', 0] = df.loc['Sum PVs', 0] + df.loc['PV terminal value', 0]
        df.loc['Net debt', 0] = parameter_list[-1]
        df.loc['Equity value', 0] = df.loc['Company value (enterprise value)', 0] - df.loc['Net debt', 0]
        equity_value = df.loc['Equity value', 0] 
        df = df.applymap(lambda x: comma_format(x))
        df = df.fillna('')
        column_name_list = range(6)
        df = df[column_name_list]
        return df, equity_value


# st.title('Monte Carlo Valuation App')

# with st.expander('How to Use'):
#     st.write('This application allows you to conduct a **probabilistic** \
#         valuation of companies you are interested in. Please enter the \
#         **stock ticker** of your company. Subsequently, the program will \
#         provide you with **historical key metrics** you can use to specify \
#         key inputs required for valuing the company of your choice. \
#         In addition, you need to provide a **discount rate** and a **terminal \
#         growth rate** at which your company is assumed to grow after year 5 \
#         into the future.')

# st.header('General company information')
# ticker_input = st.text_input('Please enter your company ticker here:')
# status_radio = st.radio('Please click Search when you are ready.', ('Entry', 'Search'))


# @st.cache
def get_company_data(ticker_input):
    company = Company(ticker_input)
    return company

ticker_input = 'aapl'
company = get_company_data(ticker_input)

before (11, 3)
after (11, 3) Index(['2020-09-30', '2021-09-30', '2022-09-30'], dtype='object')


C:\Users\Berkcan\AppData\Local\Temp\ipykernel_8144\4275074897.py:98: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prior_revenue_list.append(df.loc['Total Revenue'][i-1])


In [44]:
dff = company.inputs
dff

,2020-09-30,2021-09-30,2022-09-30,Historical average
revenueGrowth,NaN,0.332594,0.077938,2.052659e-01
ebitMargin,0.241473,0.297824,0.302887,2.807281e-01
taxRate,0.144282,0.133023,0.162045,1.464496e-01
netCapexOverSales,-0.01365,-0.000544,-0.001004,-5.065918e-03
nwcOverSales,0.051269,-0.02726,-0.053542,-9.844217e-03
netDebt,74420000000.0,89779000000.0,96423000000.0,8.687400e+10


In [42]:
dff.loc['EBIT']

2020-09-30     66288000000.0
2021-09-30    108949000000.0
2022-09-30    119437000000.0
Name: EBIT, dtype: object

In [ ]:
income_statement_list = ['Total Revenue', 'EBIT', 
        'Pretax Income', 'Tax Provision'
        ]
balance_sheet_list = ['Current Assets', 'Cash And Cash Equivalents',
'Current Liabilities', 'Current Debt',
'Long Term Debt'
]
balance_sheet_list_truncated = ['Current Assets', 'Cash And Cash Equivalents',
'Current Liabilities', 'Long Term Debt'
]
balance_sheet_list_no_debt = ['Current Assets', 'Cash And Cash Equivalents',
'Current Liabilities'
]

cash_flow_statement_list = ['Depreciation And Amortization', 
'Capital Expenditure'
]

In [ ]:
sbux = valinvest.Fundamental('SBUX')
sbux.fscore()